In [3]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("adirizq/shopee-product-images")
# print("Path to dataset files:", path)

In [4]:
%ls ../data

data_products_id_small.csv           data_products_id_tiny_with_desc.csv
data_products_id_tiny.csv            images/


In [5]:
import pandas as pd

data_path = "../data"

product_df = pd.read_csv(f"{data_path}/data_products_id_tiny.csv")
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31931 entries, 0 to 31930
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     31931 non-null  int64 
 1   image          31931 non-null  object
 2   name           31931 non-null  object
 3   shop_name      31931 non-null  object
 4   shopid         31931 non-null  int64 
 5   main_category  31931 non-null  object
 6   sub_category   31931 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.7+ MB


In [6]:
product_df.duplicated().sum()

np.int64(0)

In [7]:
product_df.isna().sum()

product_id       0
image            0
name             0
shop_name        0
shopid           0
main_category    0
sub_category     0
dtype: int64

In [8]:
product_df['main_category'].value_counts()

main_category
Olahraga & Outdoor           2825
Elektronik                   2597
Perawatan & Kecantikan       2100
Aksesoris Fashion            1800
Perlengkapan Rumah           1800
Komputer & Aksesoris         1500
Ibu & Bayi                   1500
Buku & Alat Tulis            1406
Kesehatan                    1403
Otomotif                     1400
Hobi & Koleksi               1400
Pakaian Wanita               1365
Handphone & Aksesoris        1300
Fashion Bayi & Anak          1200
Makanan & Minuman            1100
Pakaian Pria                 1086
Sepatu Wanita                1062
Tas Wanita                   1051
Fashion Muslim               1049
Tas Pria                      749
Sepatu Pria                   738
Souvenir & Party Supplies     500
Jam Tangan                    500
Fotografi                     500
Name: count, dtype: int64

In [9]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com",
)

In [10]:
def create_description(row):
    response = deepseek_client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": (
                    "Tulis deskripsi produk dalam Bahasa Indonesia. Gunakan gaya bahasa e-commerce. Tanpa markdown, tanpa HTML."
                )
            },
            {
                "role": "user",
                "content": (
                    f"Buat deskripsi untuk produk '{row['name']}' kategori '{row['main_category']}' sub-kategori '{row['sub_category']}'. "
                )
            }
        ],
        stream=False
    )

    return response.choices[0].message.content.strip()

In [11]:
product_df['description'] = ""

In [ ]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# Path to checkpoint folder
checkpoint_dir = Path("desc_checkpoints")
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# Attach cached descriptions from TXT files if available
def load_description(pid):
    path = checkpoint_dir / f"{pid}.txt"
    if path.exists():
        return path.read_text(encoding="utf-8").strip()
    return pd.NA

product_df["description"] = product_df["product_id"].map(load_description)

# Compute only missing ones and save to individual TXT files
for idx, row in tqdm(product_df.iterrows(), total=len(product_df)):
    pid = row["product_id"]
    if pd.isna(row["description"]):
        desc = create_description(row)
        product_df.at[idx, "description"] = desc

        # Save to TXT file as cache
        with open(checkpoint_dir / f"{pid}.txt", "w", encoding="utf-8") as f:
            f.write(desc)

# Optional full CSV export
product_df.to_csv("products_with_full_description.csv", index=False)

 13%|█▎        | 4029/31931 [00:15<01:45, 264.72it/s]

In [ ]:
import pandas as pd

# Function to read .txt content by product_id
def load_txt_description(pid):
    path = checkpoint_dir / f"{pid}.txt"
    if path.exists():
        return path.read_text(encoding="utf-8").strip()
    return pd.NA

# Map the descriptions into the DataFrame
product_df["description"] = product_df["product_id"].map(load_txt_description)

In [155]:
product_df

,product_id,image,name,shop_name,shopid,main_category,sub_category,description
0,19941741813,9cedf3c19821e83aa88a1e2f401e4e3b,Plac Poles sepatu Bersih Cream Pemutih Shoes S...,rejeki007,519630646,Sepatu Pria,Aksesoris & Perawatan Sepatu,Plac Poles Sepatu Bersih Cream adalah solusi p...
1,8609667001,94b8c7d8fabca08490e1398864a7e3d7,Bantalan Sepatu Depan / Insole (2pcs) - PR0015,Neofelis Houseware,113260191,Sepatu Wanita,Aksesoris & Perawatan Sepatu,Bantalan Sepatu Depan / Insole (2pcs) - PR0015...
2,845133903,882bfbef267af16d4a84624d1f33a885,MKL - Saddle Shoes / Shoee Tree Catokan Sepatu...,jakartakinesio,1317599,Sepatu Pria,Aksesoris & Perawatan Sepatu,**MKL - Saddle Shoes / Shoee Tree Catokan Sepa...
3,11448295634,2d16dcd0429f2c198fd3381d17696d5b,"ANTI JAMUR TAS KULIT DAUN TALAS ANTI AIR, LEAT...",Garael Official Store,526900249,Sepatu Pria,Aksesoris & Perawatan Sepatu,"**ANTI JAMUR TAS KULIT DAUN TALAS ANTI AIR, LE..."
4,12648185204,25eac0de61c8e0f0821daf72fb37f8c6,1 Pasang Insole Pad dan T-Shape Untuk Sepatu H...,Dishdash,14226667,Sepatu Wanita,Aksesoris & Perawatan Sepatu,Bikin pakai high heels jadi lebih nyaman sehar...
...,...,...,...,...,...,...,...,...
31926,8319206783,sg-11134201-22110-94zj010rwdkv93,SPEEDS Resistance Bands Tali Pembantu Fitness ...,CAHAYASTORE888,235433701,Olahraga & Outdoor,Yoga & Pilates,<NA>
31927,18440709211,1751fd2c7a26809c474fc53ddc2fe96c,Karet Pull Up Resistance Band Fitness Gym Late...,SportMoll,446317704,Olahraga & Outdoor,Yoga & Pilates,<NA>
31928,21726848050,ddda4b7866950ad892e17b1a8f18f050,SPEEDS Resistance Band 1 Set Yoga Band Loop Ba...,Speedsstore,184942443,Olahraga & Outdoor,Yoga & Pilates,<NA>
31929,12859437106,ac1b114802405545111f57855e7e6cf4,LIGHTSPEED RESISTANCE TUBE BAND / RESISTANCE 1...,Lightspeed Official Shop,478845512,Olahraga & Outdoor,Yoga & Pilates,<NA>
